# Opal Masterclass mit maschinellem Lernen

Lade Funktionen und alles was sonst noch nötig ist...

In [ ]:
import os
if "opal_mc_ml_helpers.py" in os.listdir():
    print("Helpers schon vorhanden")
else:
    print("Helpers nicht vorhanden, starte Download...", end="")
    os.system("wget -O opal_mc_ml_helpers.py \"https://raw.githubusercontent.com/ntiltmann/opal-mc-ml/main/opal_mc_ml_helpers.py\"")
    print(" abgeschlossen!")
from opal_mc_ml_helpers import *

Lade die Bilder der Events in eine Liste mit dem Namen bilddaten

In [ ]:
bilddaten = lade_bilder()

Mit `bilddaten[x]` kann auf das `x-1`-ste Bild zugegriffen werden. Die Methode `.zeige_bild()` gibt das Bild grafisch aus.

In [ ]:
bilddaten[53].zeige_bild(zeige_kategorie=True)

Mit `zeige_uebersicht()` erhält man eine Übersicht über die Verteilung der Kategorien. Der erste Parameter gibt an, wie die Spalte beschriftet sein soll. Danach folgen ein oder mehrere Listen mit Bildateien.

In [ ]:
zeige_uebersicht(["Roh"], bilddaten)

Nun werden die Bildateien in zwei Datensätze aufgeteilt. `tv` ist kurz für Training+Validierung. 

In [ ]:
bilddaten_tv, bilddaten_test = trenne_bilddaten_zufaellig(bilddaten, anteil_erster_block=0.3)

Im nächsten Schritt werden die `tv`-Daten vervielfältigt, damit das Training einfacher wird. Alle Bilder werden verfünffacht. Dafür werden jeweils vier Kopien des Bildes mit zufälligen Rotationen erstellt. Die Hälfte der Bilder wird zusätzlich zufällig gespiegelt. 
Um das Training der Leptonenzerfälle zu verbessern werden die Daten zu diesen Events zusätzlich noch verzwölffacht.

In [ ]:
faktor_alle = 5
faktor_leptonen = 12
bilddaten_tv_vermehrt = vervielfaeltige_daten(bilddaten_tv, [faktor_alle, faktor_alle*faktor_leptonen, faktor_alle*faktor_leptonen, faktor_alle*faktor_leptonen])

Nun werden die `tv`-Daten in Training (`t`) und Validierung (`v`) unterteilt.

Abschließend wird erneut eine Übersicht ausgegeben.

In [ ]:
bilddaten_training, bilddaten_validierung = trenne_bilddaten_zufaellig(bilddaten_tv_vermehrt, anteil_erster_block=0.66)

zeige_uebersicht(["Roh", "TV", "TV verm", "Train", "Vali", "Test"], bilddaten, bilddaten_tv, bilddaten_tv_vermehrt, bilddaten_training, bilddaten_validierung, bilddaten_test)

Die Trainingsdaten sind jetzt passend vorbereitet. Im nächsten Block wird ein Modell zum Lernen erstellt. Die Struktur wird einmal ausgegeben. In den letzten beiden Zeilen werden die Trainings- und Validierungsdaten geladen.

In [ ]:
modell = Modell()
modell.lade_modellstruktur()
modell.zeige_modelluebersicht()
modell.lade_trainingsdaten(bilddaten_training)
modell.lade_validierungsdaten(bilddaten_validierung)

Mit `trainiere()` wird das Training des Modells gestartet. `anzahl_epochen` gibt an, wie oft über alle Trainings- und Validierungsdaten gelaufen wird. Bei diesen Daten sind häufig Werte zwischen 3 und 6 sinnvoll.

In [ ]:
modell.trainiere(anzahl_epochen=4)

Die Lernkurve zeigt an, wie die Treffergenauigkeit sich im Laufe des Trainings verändert. Dabei wird die Treffergenauigkeit in Trainingsdaten und Validierungsdaten getrennt aufgetragen.

In [ ]:
modell.zeige_lernkurve()

Mit dem trainierten Modell können wir nun die Kategorien der Testdaten vorhersagen. Dabei wird die schon zugewiesene Kategorie der Testdaten ignoriert. Die Confusion Matrix ist ein Indikator dafür, wie gut die Vorhersage mit dem Modell funktioniert. Ein perfektes Modell hätte nur Einträge auf der Diagonalen: Jede Kategorie wird vom Modell so erkannt, wie sie vorher schon dem Bild zugeordnet war. Tatsächlich treten häufig einige falsche Vorhersagen auf. 

In [ ]:
bilddaten_vorhersage = modell.erstelle_vorhersage(bilddaten_test)
zeige_confusion_matrix(bilddaten_test, bilddaten_vorhersage)

Hier wird die Übersicht über die Verzweigungsverhältnisse noch um eine Spalte mit den vorhergesagten Daten erweitert.

In [ ]:
zeige_uebersicht(["Roh", "TV", "TV verm", "Train", "Vali", "Test", "Vorh"], bilddaten, bilddaten_tv, bilddaten_tv_vermehrt, bilddaten_training, bilddaten_validierung, bilddaten_test, bilddaten_vorhersage)

Man kann sich nun noch anschauen, welche Bilder falsch vorhergesagt wurden. Gelegentlich ist eine Einteilung auch für Menschen nicht eindeutig möglich.

In [ ]:
zeige_falsche_vorhersagen(bilddaten_test, bilddaten_vorhersage, anzahl=5)